This file imports an EXCEL model (here template example from BW25 sharepoint folder) and matches it with the background database in the project. For this repository we are currently working with ecoinvent310clca

After successfully importing a database to a Brightway project, the database is stored in the project and doesn't have to be imported every time we open the repository. Unless we make changes to the background database of course

# EXCEL Model importer

#### 1. Import packages

In [37]:
# basic imports from brightway
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
from bw2data import databases
import bw2io as bi
from bw2io import ExcelImporter
from bw2io.importers import SingleOutputEcospold2Importer
import bw2analyzer as bwa
from bw2data import methods
import argparse
import bw2data as bd
import os

# other relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

#### 2. Set project and see databases
The databases are required here, because the Excel database we want to import in this script needs to be matched to these.

In [38]:
# define a project where we install the databases and work in this script
bd.projects.set_current('LCA_Toolbox')

In [39]:
bd.databases

Databases dictionary with 5 object(s):
	bafu
	biosphere3
	ecoinvent-3.12-biosphere
	ecoinvent-3.12-consequential
	example_db

In [ ]:
el_bio3 = bd.Database('biosphere3')

#### 3. Import the EXCEL file containing the model 
The file import is easy, we just use the Excel importer function from bw2io as can be seen below.

In [ ]:
# Get user's home directory (C:\Users\USERNAME)
home = os.path.expanduser("~")

In [ ]:
# Here you need to change the path to your local path where you have stored the excel file, this can also be on your sharepoint. 

excel_db = bi.ExcelImporter(os.path.join(
    home,
    "OneDrive - 2.-0 LCA Consultants ApS",  # This stays the same for all users
    "Intranet - Brightway2",
    "excel_model_example",
    "example_db.xlsx"
))

#### 4. Importing means matching with background databases and writing the database
The part that is a bit trickier to understand is how the matching with the relevant background databases works. In essence, you need to match the newly imported databases with all background databases used in the model.

In [ ]:
excel_db.apply_strategies()

In [ ]:
excel_db.match_database(fields=['name','location'])

In [ ]:
excel_db.match_database(db_name='ecoinvent-3.12-consequential',
                        fields=['name','location','unit'])

In [ ]:
excel_db.match_database(db_name='ecoinvent-3.12-biosphere',
                        fields=['name','categories','unit'])

In [ ]:
excel_db.match_database(db_name='biosphere3',
                        fields=['name','categories','unit'])

In [ ]:
# check foreground imports for unlinked processes
pd.DataFrame(excel_db.unlinked)

In [ ]:
# if there are unlinked exchanges, this helps localizing them in order to fix them
list(excel_db.unlinked)


In [ ]:
excel_db.write_database()

#### 5. Store data in a dataframe so it can be analysed in Brightway 2.5
This step allows you to analyse the imported database in the Brightway framework.

In [ ]:
db_template = bd.Database('example_db')


In [ ]:
bd.databases

In [ ]:
data_db_template = [activity.as_dict() for activity in db_template]


In [ ]:
df_db_template = pd.DataFrame(data_db_template)


In [40]:
from IPython.display import display
display(df_db_template)

,comment,location,production amount,reference product,unit,name,worksheet name,database,code,type,id
0,This dataset represents production of porcelai...,RER,1,porcelain ceramics production,kilogram,porcelain ceramics production,LCI,example_db,a1b7153986d8f8387752b14278a0b4c2,processwithreferenceproduct,270173767135997952


In [41]:
# what also works to show all activities in a database is the following
for act in db_template:
    print(act)

'porcelain ceramics production' (kilogram, RER, None)
